<img style="border-radius: 0.5rem;" src="banner.jpg" />
<a style="margin-top: 1rem;" class="btn btn-lg btn-block btn-success" href="https://dsg.ucfsigai.org/fa18/signin/">
    Sign in (https://dsg.ucfsigai.org/fa18/signin/)
</a>

# Solving the Computationally Impossible with Heuristics
---
by: Evan Waldmann \([@Waldmannly](github.com/Waldmannly/)\), on 10 Oct 2018

Optimization techniques usually rely on have a nice, continuous function that is capable of being minimized through derivative based techniques such as gradient descent. 

So, say we wanted to minimize a simple function like f(x) = x^4 + x^2 -4 x^2 + 10. Optimization methods basically do what you used to do when you took calculus. You find d/dx(f(x)) solve for x and plug the answer back into f(x). 

So d/dx(f(x)) = 4x^3 + 2x - 8 x = x^2 (4x - 6). Then we have 

x^2 =0 and 4x-6 = 0. So our x is 0 and +- sqrt(3/2). 

then we plug in f(x) and get (0,10), (-3/2, 31/4), (3/2, 31/4). So we technically have two global minima. 



But what do we do for real world problems that can not be boiled down to a nice differentible function? 

Well, we end up guessing, which works when we don't care if we have the exact solution (i.e. the absolute minimum). 

If the problem if simple enough, we usually just test all of the possible scenarios and take the best value as our answer. Since computes are super fast, this often works well because its easy. 

BUT when the problems are not simple, we have to make our computers make smarter guesses. 

Let's look at a classic problem called the Traveling Salesman Problem that has been around since the 1930's. 


TALK ABOUT TSP problem 


If you were to brute force by randomly guessing then running time would end up being O(n!), so for any number of cities over say 20 it would be unrealistic. 




So what are some ideas that you have for making your computer guess a bit smarter. (Those of you done with CS1 or CS2 should have a few ideas). 


Well most of those answers can be boiled down to either 
 (1) Exploration  - looking around the solution space to gather more information
 (2) Exploitation - Making the best descision given the current information 
 
I am not going to dive too deep into the the theory here, because it does get dense and statistically hard very quickly. The idea is that we need to explore in order to make sure that we did not miss anything that could be hiding in the solution space, but we also need to exploit what we know so that we can get as close as we can to the right solution. The problem is that these to idea are competing with each other, often if we are exploring we can not exploit and if we are exploiting we can not explore. The balance of these two ideas is pretty much what the field of heuristics is all about (and the "correct" answer often depends on the solution you are trying to solve).  
 
 
 
So the key take away is that  heuristics is about smart guessing and by either exploring the solution space or exploiting what we know, we can help our computer guess a bit better.

 

In [2]:
def dataset(path):
    import os
    from pathlib import Path
    datadir = Path(os.environ["DATA_DIR"])
    return Path(datadir.joinpath(path))


One of the most important things in heuristics is how you encode your problem into your code. 
It is important to recognize that almost every solution will be encoded differently, so you can think of these heuristic algorithms as a framework that you can shape to fit your problem. 

For the traveling sales man problem our solution will be a "tour" of cites. We will represent this as a vector of numbers [2 4 3 1 6 5], where this vector will mean that our tour travels from 2 to 4 to 3 and so on. 

Since the problem is about finding the minmum distance to tour all of the cities, we need a cost function that calculates the distance for a given vector. 

We also need to figure out how to guess solutions that should be close to some given solution. This make our guessing less random. 

With that we can define the traveling sales man random neighboring solution generator and solution cost function below. 

In [148]:
import numpy as np 

def tspobjective(a, d):
    ofv = 0
    for i in range(len(a)-1):
        ofv = ofv + d[a[i]][a[i+1]]
    ofv = ofv +  d[a[len(a)-1]][a[1]]  
    return ofv;

a = [4 ,2 ,6 ,1 ,7 ,5 ,3]
d = [[0,   6, 42, 29,  5, 13, 25, 44, 44,  8],[6 ,  0, 44, 44, 22, 37,  7, 19, 42, 36 ],[42 ,44,  0, 44, 30,  2, 39, 43, 31, 35 ], [29, 44 ,44,  0 ,34, 18 ,30,  8, 32,  2 ],[ 5, 22 ,30, 34 , 0 ,13 , 3 , 5, 38, 32 ], [13, 37  ,2, 18 ,13 , 0 ,15 ,43,  2, 20 ],[25,  7 ,39, 30 , 3 ,15 , 0 ,18, 35, 36 ],[44, 19 ,43,  8  ,5 ,43 ,18 , 0,  9, 23 ],[44, 42 ,31, 32 ,38 , 2 ,35 , 9,  0, 21 ], [8, 36, 35 , 2, 32 ,20, 36 ,23, 21,  0 ]]


print(tspobjective(a,d))


randomperm = np.random.permutation(7)
print(randomperm)
print(tspobjective(randomperm,d))


200
[2 3 5 1 0 4 6]
143


In [65]:
a = np.random.permutation(7)
ofv = tspobjective(a,d)
bestofv = ofv
bestTour = a 

for j in range(10):
    a = np.random.permutation(7)
    ofv = tspobjective(a,d)
    if (ofv < bestofv):
        bestofv = ofv 
        bestTour = a 
    
print(bestofv)
print(bestTour)




124
[3 4 1 0 2 5 6]


In [150]:
import random 

def tspswap(a):
    l = random.randint(0,len(a)-1)
    m = random.randint(0,len(a)-1)
    while l==m:
        m = random.randint(0,len(a)-1)
    temp = a[l]
    a[l] = a[m]
    a[m] = temp
    newa = a
    return(newa); 

a = [4 ,2 ,6 ,1 ,7 ,5 ,3]
print(a)
print(tspswap(a))
print(tspswap(a))

[4, 2, 6, 1, 7, 5, 3]
[4, 2, 6, 1, 3, 5, 7]
[4, 1, 6, 2, 3, 5, 7]


In [275]:
a = np.random.permutation(7)
bestofv = tspobjective(a,d)
best=a[:]

for i in range(1,10):  
    #generate neighbor sols
    na= tspswap(a)
    naofv = tspobjective(na,d)
    if (naofv<bestofv):
        best = na[:]
        bestofv = naofv
    a = na[:] #create neighbors from the neighbor to get variety 
    
Best = best 
BestOFV = bestofv 

print(Best)
print(bestofv)

[2 5 0 3 1 6 4]
95


In [ ]:
a = [4 2 6 1 7 5 3]
d = [[0   6 42 29  5 13 25 44 44  8 ]
     [6   0 44 44 22 37  7 19 42 36 ]
     [42 44  0 44 30  2 39 43 31 35 ]
     [29 44 44  0 34 18 30  8 32  2 ]
     [ 5 22 30 34  0 13  3  5 38 32 ]
     [13 37  2 18 13  0 15 43  2 20 ]
     [25  7 39 30  3 15  0 18 35 36 ]
     [44 19 43  8  5 43 18  0  9 23 ]
     [44 42 31 32 38  2 35  9  0 21 ]
      [8 36 35  2 32 20 36 23 21  0 ]]



Heuristics are usually based off of processes in nature and the first algorithm I am going to cover is called Simulated Annealing.

The idea for simulated Annealing came from the process of hardening metal. In order to harden metal you heat it up and left it cool -- its called annealing. 

So how exactly does that relate to smart guessing? 

Well when you heat up the metal to when it is glowing read, the particles in the metal are bouncing around a lot, and as the metal slowly cools, the particles start to move less and less. This gives us a frame work for how to guide our exploration versus exploitation. When the metal is super hot, we are going to explore the solution space more, and as we cool, we are going settle in on specific spots in the space that have produced good results. 


In [274]:
import random 
import math 

def SA(a,d, T,tf, N, K, alpha):
    currenta = a[:]
    besta = a[:]
    bestofv = tspobjective(a,d)
    ofvNew = bestofv
    ofvOld = bestofv
    na =currenta[:]
    #initalize counting variables 
    n=1
    k=1
    kk=0
    iteration =-1
    x= [0 for x in range(100)]
    #outer loop
    while tf<T:
        iteration = iteration +1
        #reset counting variables
        n=1
        k=1
        #inner loop
        while n<= N and k<=K:
            #generate neighbor solution 
            na= tspswap(currenta[:])[:]
            kk = kk+1 #summation of total number of solution generated 
            #store values so you only have to compute them once
            ofvNew = tspobjective(na, d)
            ofvOld = tspobjective(currenta,d)
            if (ofvNew < ofvOld): #check to see if neighbor solution is better 
                currenta = na[:]
                n=n+1
                k=k+1
                if ofvNew < bestofv: #check if neighbor solution is best 
                    bestofv = ofvNew
                    besta = currenta[:]
            elif random.uniform(0, 1) < math.exp(-(ofvNew -ofvOld)/T):#check to accept bad solution
                k=k+1
                n=n+1
                currenta= na[:]
            else:
                n=n+1
        T= alpha*T #decrease Temperature 
        x[iteration] = bestofv
    return(besta, bestofv, kk,iteration,x); 
    
N=20
K=5 #stopping condition for inside loop
T=10000
tf =4 #stopping condition for outer loop 
alpha = .9 #rate of cooling 
a = [1,2,3,4,5,6]
print(SA(a, d, T, tf, N,K,alpha))


([2, 5, 4, 6, 1, 3], 87, 574, 74, [120, 120, 120, 120, 120, 120, 120, 120, 120, 118, 118, 118, 118, 106, 106, 106, 106, 106, 106, 98, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 90, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 87, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


We will finish up this lecture by cover Particle Swarm Optimization (PSO). 


In [341]:
#Set algorithm parameters (Num_particles, Max_time _step, w, c 1 , c 2 ).tstep=1.
import math 
num_particles= 4
max_time_step = 5
w = 2 # number of chances to have a neighbor solution to the particles current solution
c1 = 2# number of chances to have a neighbor solution to the particles best solution
c2 = 1 # number of chances to have a neighbor solution to the global best solution 
tstep = 1
numcites =7

#Generate initial particles.
particles = [np.random.permutation(numcites) for particles in range(num_particles)]
ofvs = [-1 for ofvs in range(num_particles)]
for i in range(0,num_particles):
    ofvs[i] = tspobjective(particles[i],d)


print(particles)
print(ofvs)

#set up variables for tracking particle and global bests 
globalbestofv = min(ofvs)
globalbest = particles[ofvs.index(min(ofvs))]
print(globalbest)
print(globalbestofv)
particlesbest = particles[:]
particlesbestofv = ofvs[:]

# run the PSO algorithm 
while tstep<= max_time_step:
    #• For each particle
    for p in range(0,num_particles-1):
        #• Construct a chance box
        x = [-1 for x in range(numcites)]
        chanceBox = [ x for chanceBox in range(w+c1+c2)]
        #• Generate w neighbour solutions from selected particle and put them in the chance box.
        for i in range(0,w): 
            chanceBox[i] = tspswap(particles[p])
        #• Generate c 1 neighbour solutions from corresponding particle’s pbest and put them in the chance box.
        for i in range(w,w+c1): 
            chanceBox[i] = tspswap(particlesbest[p])
        # Generate c 2 neighbour solutions from the gbest and put them in the chance box.
        for i in range(w+c1, w+c1+c2):
            chanceBox[i] = tspswap(globalbest)
        # Select a solution from the chance box randomly
        particles[p] = chanceBox[random.randint(0,w+c1+c2-1)]#• Consider the selected solution as the new position of the selected particle
        #• Calculate the objective function value of all particles
        ofvs[p] = tspobjective(particles[p] ,d)
        #update pbest if needed
        if (ofvs[p] < particlesbestofv[p]):
            particlesbestofv[p] = ofvs[p]
            particlesbest[p] = particles[p] 
    #update gbest if needed 
    globalbestofvTemp = min(ofvs)
    if (globalbestofvTemp < globalbestofv):
        globalbest = particles[ofvs.index(min(ofvs))]
        globalbestofv = globalbestofvTemp
    tstep =tstep+1

[array([4, 0, 5, 2, 3, 6, 1]), array([0, 1, 3, 5, 6, 4, 2]), array([6, 5, 4, 1, 2, 3, 0]), array([2, 1, 6, 5, 4, 3, 0])]
[107, 160, 180, 148]
[4 0 5 2 3 6 1]
107


In [339]:
print(globalbest)
print(globalbestofv)

[2 4 3 6 0 1 5]
101
